# **Lab 3: Iterative methods**
**Kristoffer Almroth**

# **Abstract**

Third lab in the course DD2363 Methods in Scientific Computing. This lab is about iterative methods for solving the equation Ax=b and f(x)=0.

# **Set up environment**

Dependencies needed for running the code.

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import sympy as sp
from sympy import *
import random

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

import unittest

# **Introduction**

One important function of matrices is to be able to solve the equation Ax=b. This equation can be solved by a variety of different methods, imcluding direct methods and iterative methods. In this lab three iterative methods will be implemented: Jacobi iteration, Gauss-Seidel iteration and GMRES. Additionally, Newton's method for scalar nonlinear equations will be implemented.

# **Methods**

This section is based on the theory from the fourth and fifth lecture and the lecture notes. Some of the algorithms are based on the pseudo code described in the lecture notes. All code is tested using various different test cases, including the test cases given in the lab requirements and some additional tests. 

Since floating point precision were used in the tests, there could be rounding errors.  The test methods of the library [numpy](https://docs.scipy.org/doc/numpy/reference/routines.testing.html) is used to test equality to a certain precision.


## Iterative methods for $Ax=b$

### Jacobi iteration

Jacobi iteration works through matrix splitting, where $A = A_1 + A_2$. $A_1$ is the diagonal elements of $A$ while $A_2 = A - A_1$.

$x^{(k+1)} = (I-D^{-1}A)x^{(k)} + D^{-1}b$

In [0]:
def jacobi(A, b, TOL):

  n = A.shape[0]
  I = np.identity(n)
  D = np.diag(np.diag(A))
  D_inv = np.linalg.inv(D)
  c = D_inv.dot(b)
  x = c

  while np.linalg.norm(A.dot(x) - b) > TOL:
    x = (I-np.matmul(D_inv,A)).dot(x) + c

  return x

### Gauss-Seidel iteration

Gauss-Seidel iteration also works through matrix splitting, but instead of a diagonal matrix, $A_1 = L$ where L is a lower triangluar matrix.

$x^{(k+1)} = (I-L^{-1}A)x^{(k)} + L^{-1}b$

In [0]:
def GaussSeidel(A, b, TOL):

  n = A.shape[0]
  I = np.identity(n)
  L = np.tril(A)
  L_inv = np.linalg.inv(L)
  c = L_inv.dot(b)
  x = c

  while np.linalg.norm(A.dot(x) - b) > TOL:
    x = (I-np.matmul(L_inv,A)).dot(x) + c

  return x

### Test cases

A requirement for convergence is that $||I-\alpha A|| < 1$. Randomly generated test data does not usually fill this requirement, so a precondition is used. $BAx=Bb$ where B is close to the inverse of A makes the randomly generated matrix always converge.

In [124]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,1000):
      row = randrange(30) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)

      # Preconditioning for convergence
      B = np.linalg.inv(A)
      B = B * 0.8
      A = np.matmul(B,A)
      b = B.dot(b)

      x2 = jacobi(A,b,1e-7)
      x3 = GaussSeidel(A,b,1e-7)

      # x = A^(-1)b
      np.testing.assert_allclose(x2, x, rtol=1e-5, atol=0)
      np.testing.assert_allclose(x3, x, rtol=1e-5, atol=0)

      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=5)
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x3)-b), 0, decimal=5)

      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=5)
      np.testing.assert_almost_equal(np.linalg.norm(x-x3), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 1.219s

OK


## Newton's method for scalar nonlinear equation

Newton's method is based on the first order derivative of the nonlinear function.

$x^{(k+1)} = x^{(k)} - f^{'}(x^{(k)})^{-1} f(x^{(k)})$

In [0]:
def NewtonsMethod(f, y, TOL):

  # Using sympy to calculate f'
  x = Symbol('x')
  fprime = f.diff(x)
  f = lambdify(x, f)
  fprime = lambdify(x, fprime)

  while np.abs(f(y)) > TOL:
    y -= f(y)/fprime(y)

  return y

In [128]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,1000):

      # Generating a random function using sympy
      exponents = [random.randint(1,10) for i in range(3)]
      x = Symbol('x')
      f = x**exponents[0] + x**exponents[1] + x**exponents[2] - random.randint(0,100)

      y = NewtonsMethod(f, 1, 1e-7)
      f = lambdify(x, f)

      # |f(x)|
      np.testing.assert_almost_equal(np.abs(f(y)), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 4.189s

OK


## GMRES method

Generalised minimal residual, where we want to calculate $\displaystyle \min_{y \in \mathbb{R}^k} ||b-AQ_ky||$ where $Q_k$ is an orthonormal basis for $K_k$. Through Arnoldi iteration we get $\displaystyle \min_{y \in \mathbb{R}^k} || \space ||b||e_1-H_ky||$.

We get the approximation through $x^{(k)} = Q_ky$

In [0]:
def arnoldiIteration(A, b, k):
  n = A.shape[0]
  Q = np.zeros(shape=(n,k+1))
  H = np.zeros(shape=(k+1,k))
  v = np.zeros(n)

  Q[:,0] = b / np.linalg.norm(b)
  for j in range(0,k):
    v = A.dot(Q[:,j])
    for i in range(0, j):
      H[i,j] = Q[:,i].dot(v)
      v = v - H[i,j] * Q[:,i]
    H[j+1,j] = np.linalg.norm(v)
    Q[:,j+1] = v / H[j+1,j]
  
  return Q, H

def gmres(A, b, TOL):

  n = A.shape[0]
  Q = np.zeros(shape=(n,n))
  x = np.zeros(n)
  k = 1
  r = b * 2

  Q[:,0] = b / np.linalg.norm(b)

  while np.linalg.norm(r) / np.linalg.norm(b) > TOL:
    (Q, H) = arnoldiIteration(A, b, k)

    e_k = np.zeros(k+1)
    e_k[0] = np.linalg.norm(b)
    y = np.linalg.lstsq(H, e_k)[0]

    r = H.dot(y)
    r = -r
    r[0] += np.linalg.norm(b)

    k = k+1  

  # Q matrix is of wrong size, downscale it
  Q = np.delete(Q, k-1, 1)

  x = (Q).dot(y)

  return x;

In [131]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,2):
      row = randrange(10) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)
      x2 = gmres(A,b,1e-6)

      # x = A^(-1)b
      np.testing.assert_allclose(x2, x, rtol=1e-2, atol=0)

      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=2)

      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=2)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
.
----------------------------------------------------------------------
Ran 1 test in 42.145s

OK


# **Results**

The iterative methods Jacobi iteration, Gauss-Seidel iteration and Newton's method were accurate to more than five decimals when the methods were called with the precision $1e-7$.

GMRES were accurate up to the second decimal when called with the precision $1e-6$. Each test took approximataly six seconds, which is slow.

# **Discussion**

I find it weird that the dimensions of Q and y in the GMRES functions did not align, which was solved by removing the outer column in the Q matrix. Based on the specification of numpy least square algorithm, y should be of size k, which does not match with Q that is of size $n \space x \space k+1 $. The solution $x$ still seems to converge to the correct value, so this potential error is acceptable.